#### Adicionais deste script:
- Analisar o script prepare_sequences
- Talvez colocar script para preparação de conjuntos
- Usar rede LSTM certo!
- Colocar todos os sensores > D4 e D5 possuem NaN

In [1]:
# Libraries
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Carregando dados
d1 = np.load('Dados_1.npy'); d1 = d1[:,0:200] # eliminar NaN
d2 = np.load('Dados_2.npy'); d2 = d2[:,0:200] # eliminar NaN
d3 = np.load('Dados_3.npy'); d3 = d3[:,0:200] # eliminar NaN
d4 = np.load('Dados_4.npy'); d4 = d4[:,0:200] # eliminar NaN
d5 = np.load('Dados_5.npy'); d5 = d5[:,0:200] # eliminar NaN
cl = np.load('Classes.npy', allow_pickle=True)
labels = np.array([])
num_samples = 50000
for ind in range(num_samples):
    if cl[ind,0]=='Classe A':
        labels=np.append(labels,0)
    if cl[ind,0]=='Classe B':
        labels=np.append(labels,0.25)
    if cl[ind,0]=='Classe C':
        labels=np.append(labels,0.50)
    if cl[ind,0]=='Classe D':
        labels=np.append(labels,0.75)
    if cl[ind,0]=='Classe E':
        labels=np.append(labels,1)
labels = np.transpose(labels)
# Aplicando MinMaxScaler
scaler = MinMaxScaler()
data = np.concatenate((d1, d2, d3), axis=1)
scaled_data = scaler.fit_transform(data)

In [2]:
# script para preparar as sequencias
def split_sequence(data, n_steps):
    x, y = list(), list()
    for i in range(len(data)):
        end_ix = i + n_steps
        if end_ix > len(data)-1:
            break
        seq_x, seq_y = data[i:end_ix], data[end_ix]
        x.append(seq_x)
        y.append(seq_y)
    return np.array(x), np.array(y)

In [3]:
from sklearn.model_selection import train_test_split
n_steps = 10
x, y = split_sequence(scaled_data, n_steps)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2) #, random_state = 42)

In [4]:
from keras.models import Sequential  
from keras.layers import *  

modelo = Sequential([
    LSTM(50, activation='relu', input_shape=(n_steps, 1)),
    Dense(1)])
modelo.compile(optimizer='adam', loss='mse')
history = modelo.fit(x_train, y_train, epochs=5, batch_size=32)

2024-02-11 20:22:29.824902: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-11 20:22:43.527057: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-11 20:22:43.527177: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-11 20:22:46.205512: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-11 20:22:52.071566: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-11 20:22:52.092415: I tensorflow/core/platform/cpu_feature_guard.cc:1

NameError: name 'model' is not defined

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)


In [ ]:
# Conjunto de treino e teste
num_samples = 50000
split = int(0.8 * num_samples)
train_d1, test_d1 = d1_scaled[:split,0:200], d1_scaled[split:,0:200]
train_d2, test_d2 = d2_scaled[:split,0:200], d2_scaled[split:,0:200]
train_d3, test_d3 = d3_scaled[:split,0:200], d3_scaled[split:,0:200]
train_d4, test_d4 = d4_scaled[:split,0:200], d4_scaled[split:,0:200]
train_d5, test_d5 = d5_scaled[:split,0:200], d5_scaled[split:,0:200]

train_labels, test_labels = labels[:split], labels[split:]

train_d = np.concatenate((train_d1, train_d2, train_d3), axis=1)
test_d  = np.concatenate((test_d1, test_d2, test_d3), axis=1)
#np.isnan(np.min(d5))

In [ ]:
n_steps = 10
x, y = prepare_sequences(scaled_data, n_steps)

In [ ]:
from keras.models import Sequential  
from keras.layers import *  

modelo = Sequential()
modelo.add(Dense(units=10,input_shape=(600,))) #hidden layer 1 with input
modelo.add(Dense(units=1)) #output layer

# Compilar o modelo
modelo.compile(optimizer='adam', loss='mse')
# Treinar o modelo
history = modelo.fit(train_d, train_labels, epochs = 50, batch_size = 32, validation_split=0.2)

In [ ]:
# Vou propor uma rede LSTM
from keras.models import Sequential  
from keras.layers import *  
n_steps=10
modelo = Sequential([
    LSTM(50, activation='relu', input_shape=(n_steps, 1)),
    Dense(1)])


#modelo.add(Dense(units=10,input_shape=(600,))) #hidden layer 1 with input
#modelo.add(Dense(units=1)) #output layer

# Compilar o modelo
modelo.compile(optimizer='adam', loss='mse')

# Treinar o modelo
history = modelo.fit(train_d, train_labels, epochs = 3, batch_size = 32,
                    validation_split=0.2)